# RAG Azure AI Agent -palvelun ja Semantic Kernelin avulla

Tämä koodiesimerkki näyttää, kuinka luoda ja hallita Azure AI -agenttia hakupohjaisen generoinnin (RAG) toteuttamiseksi käyttämällä `Azure AI Agent Service` -palvelua ja `Semantic Kernel` -kirjastoa. Agentti käsittelee käyttäjän kyselyitä haetun kontekstin perusteella ja tarjoaa tarkkoja vastauksia sen mukaisesti.


SQLite-version korjaus  
Jos kohtaat virheen:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Poista tämän koodilohkon kommentointi muistikirjasi alussa:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Pakettien tuonti
Seuraava koodi tuo tarvittavat paketit:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Hakemukseen perustuva generointi Semantic Kernelin ja Azure AI Agent Servicen avulla

Tämä esimerkki näyttää, kuinka käyttää **Azure AI Agent Service** -palvelua suorittamaan **hakemukseen perustuvaa generointia (RAG)** yhdistämällä kielimalli ja ladatun dokumentin tarjoama alakohtainen konteksti.

### Miten se toimii

1. **Dokumentin lataus**: Markdown-tiedosto (document.md), joka sisältää tietoa (Contoson matkavakuutuskäytäntö), ladataan agenttipalveluun.

2. **Vektorivaraston luominen**: Dokumentti indeksoidaan vektorivarastoon, jotta sen sisältöä voidaan hakea semanttisesti.

3. **Agentin konfigurointi**: Agentti luodaan käyttämällä `gpt-4o`-mallia seuraavilla tiukoilla ohjeilla:
   - Vastaa kysymyksiin vain dokumentista haetun sisällön perusteella.
   - Kieltäydy vastaamasta, jos kysymys ei kuulu dokumentin aihepiiriin.

4. **Tiedostonhakutyökalun integrointi**: `FileSearchTool` rekisteröidään agentille, jolloin malli voi haun aikana etsiä ja noutaa asiaankuuluvia katkelmia indeksoidusta dokumentista.

5. **Käyttäjän vuorovaikutus**: Käyttäjät voivat esittää kysymyksiä. Jos dokumentista löytyy asiaankuuluvaa tietoa, agentti tuottaa perustellun vastauksen.  
   Jos ei, agentti vastaa selkeästi, että dokumentti ei sisällä riittävästi tietoa.


### Päätoiminto



Varmista ensin, että suoritat `az login` Azure CLI:llä, jotta oikea autentikointikonteksti on käytettävissä `DefaultAzureCredential`-tunnistautumisen aikana. Azure AI Agent Service ei käytä API-avaimia.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

## Johdanto

Tervetuloa tähän oppaaseen! Tässä asiakirjassa käsitellään, kuinka voit käyttää @@INLINE_CODE_0@@ tehokkaasti. Seuraavat osiot tarjoavat yksityiskohtaisia ohjeita ja esimerkkejä.

[!NOTE] Tämä opas olettaa, että sinulla on perustiedot @@INLINE_CODE_1@@:sta.

---

## Asennus

Ennen kuin aloitat, varmista, että järjestelmäsi täyttää seuraavat vaatimukset:

1. @@INLINE_CODE_2@@ on asennettu.
2. Sinulla on pääsy @@INLINE_CODE_3@@:aan.

Asennusohjeet:

1. Lataa tarvittavat tiedostot osoitteesta @@INLINE_CODE_4@@.
2. Suorita seuraava komento:

   ```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```

3. Tarkista, että asennus onnistui suorittamalla:

   @@CODE_BLOCK_1@@

[!TIP] Jos kohtaat ongelmia, tarkista @@INLINE_CODE_5@@-dokumentaatio.

---

## Käyttö

Kun asennus on valmis, voit aloittaa @@INLINE_CODE_6@@:n käytön. Alla on esimerkki peruskäytöstä:

@@CODE_BLOCK_2@@

[!IMPORTANT] Muista korvata @@INLINE_CODE_7@@ omilla arvoillasi.

---

## Vianmääritys

Jos kohtaat ongelmia, kokeile seuraavia ratkaisuja:

- Tarkista @@INLINE_CODE_8@@-asetukset.
- Suorita seuraava komento saadaksesi lisätietoja:

  @@CODE_BLOCK_3@@

[!WARNING] Älä muuta @@INLINE_CODE_9@@-tiedostoa suoraan, ellei se ole ehdottoman välttämätöntä.

---

## Yhteenveto

Tässä oppaassa käsiteltiin @@INLINE_CODE_10@@:n asennus, käyttö ja vianmääritys. Jos tarvitset lisätietoja, tutustu viralliseen dokumentaatioon osoitteessa @@INLINE_CODE_11@@.

Kiitos, että käytit tätä opasta!



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
